In [1]:
api_key = "RGAPI-6adfe7bd-a7a9-42f9-b5ed-87ea29024f5f"
my_list = [('TheK1ngPotato', 'GOLD', 3, 11), ('Draeknof', 'GOLD', 3, 0), ('Haukeas Grémory', 'GOLD', 4, 0), ('Blarckful', 'SILVER', 2, 98), ('SaintLoutr', 'SILVER', 3, 0), ('Caio73', 'BRONZE', 1, 67), ('ThePr1ncePotato', 'BRONZE', 2, 47), ('7689000', 'BRONZE', 4, 0)]

for i in range(len(my_list)):
    number = my_list[i][2]
    n = number + 10
    my_list[i] = (my_list[i][0], my_list[i][1], n, my_list[i][3])

print(my_list)

[('TheK1ngPotato', 'GOLD', 13, 11), ('Draeknof', 'GOLD', 13, 0), ('Haukeas Grémory', 'GOLD', 14, 0), ('Blarckful', 'SILVER', 12, 98), ('SaintLoutr', 'SILVER', 13, 0), ('Caio73', 'BRONZE', 11, 67), ('ThePr1ncePotato', 'BRONZE', 12, 47), ('7689000', 'BRONZE', 14, 0)]


In [2]:
import requests

version_patch = requests.get('https://ddragon.leagueoflegends.com/api/versions.json').json()
version_patch = version_patch[0]
data_champion = requests.get('https://ddragon.leagueoflegends.com/cdn/'+ version_patch +'/data/fr_FR/champion.json').json()
id_champion = data_champion["data"]

def getChampionInfo(id):
    global id_champion
    for key in id_champion:
        if id_champion[key]['key'] == str(id):
            return key
    return False

getChampionInfo(555)

'Pyke'

In [3]:
#converter roman number to int (the date won't go further 5 as there is only 5 tier possible in the data)
class roman_to_int(object):
   def romanToInt(self, s):
      """
      :type s: str
      :rtype: int
      """
      roman = {'I':1,'V':5}
      i = 0
      num = 0
      while i < len(s):
         if i+1<len(s) and s[i:i+2] in roman:
            num+=roman[s[i:i+2]]
            i+=2
         else:
            num+=roman[s[i]]
            i+=1
      return num


In [4]:
## function ranking_information : take a list of summoners name and return a list of those summoners but with their rank informations (if the summoners is not found he is not added to the output list)

def ranking_information (summoner_list_name:list):
    global list_summoner_rank
    list_summoner_rank = []

    for summoner in summoner_list_name:
    
        ## retrieve two important informations (the id and puuid) for the next steps
        
        data_summoner = requests.get('https://euw1.api.riotgames.com/lol/summoner/v4/summoners/by-name/'+ summoner +'?api_key='+api_key).json()
        if data_summoner is not None and "id" in data_summoner:
            id_summoner = data_summoner.get("id")
            puuid_summoner = data_summoner.get("puuid")
            print(id_summoner)
          
            ## with the id we can retrieve the information of the account (tier, LP, rank)  
            
            data_rank = requests.get('https://euw1.api.riotgames.com/lol/league/v4/entries/by-summoner/'+ id_summoner +'?api_key='+api_key).json()
        
            ## we check from the data_rank if the account has done any SOLO Q games for this season only !!
            
            presence_of_soloq_data = False
            for data_SOLOQ in data_rank:
                if data_SOLOQ['queueType'] == "RANKED_SOLO_5x5":
                    print(summoner,data_SOLOQ['tier'],data_SOLOQ['rank'],data_SOLOQ['leaguePoints'], "LP")
                    rank = roman_to_int().romanToInt(data_SOLOQ['rank'])
                    
                    ## here we save the informations for each account            
                      
                    list_summoner_rank.append((summoner,data_SOLOQ['tier'],rank,data_SOLOQ['leaguePoints']))
                    presence_of_soloq_data = True
            if not presence_of_soloq_data:
                print(f"No data / No SOLO Q for this account : {summoner}")

                ## we still keep the summoner in the final list (to avoid users thinking that the account was not checked)
                    
                list_summoner_rank.append((summoner,"none","0", "0"))
        else:
            print(f"Failed to retrieve data for summoner {summoner}")
            
summoner_list = ["SaintLoutr","Draeknof","Haukeas Grémory","Caio73","charlos92604","7689000","Alrsay","TheK1ngPotato","Blarckful"]
#summoner_list = ["SaintLoutr","TheK1ngPotato","Draeknof"]

ranking_information(summoner_list)

6Qf7TxMUssidN7tK57LQLfwFP-0KbGnpl0qeRHT92kQoL0i9hUA1gyZoig
SaintLoutr SILVER IV 81 LP
SJkkzi6dOPuhx-azWFVuFf8DhQPKXndDOxSPpO4jE8Icl9lR
Draeknof GOLD IV 75 LP
BpDM3cZDdKCb_xMKjMOYuph42v3BbB1hTE6h8wvMcia-xcPQ
Haukeas Grémory GOLD IV 0 LP
9hyU9dTr7a7BNTdRqIxMU45ClNmE1JiFZ7iyUlg0kmpsrhE
Caio73 BRONZE I 100 LP
Failed to retrieve data for summoner charlos92604
RvdvKpOs3UflgNFTxpKoNOJh_Ri_c_ct8cpZjAiujTpshJU
7689000 BRONZE IV 0 LP
mVi65REkPIOmCREZUyYz8rupND5i4-YqkZqb05t_spaamR0
Alrsay PLATINUM III 27 LP
Lk2rpY4KWIS8VvZuPLoxRUdqDsS1f4R8mBuwS4sOYVKbu_w
TheK1ngPotato GOLD II 38 LP
gIFdH50U0kpetV2XKoLOXk2ixYeKEE7e1Hiy7R2wV0DDn9g
Blarckful SILVER I 26 LP


In [5]:
import datetime
import calendar

## set the time of today

now = datetime.datetime.now()

start_of_day = datetime.datetime(
    year=now.year,
    month=now.month,
    day=now.day,
    hour=0,
    minute=0,
    second= 1,
)

end_of_day = datetime.datetime(
    year=now.year,
    month=now.month,
    day=now.day,
    hour=23,
    minute=59,
    second=59,
)

## retrieve the start of the day and the end of the day

end_of_day_epoch = str(int(end_of_day.timestamp()))
start_of_day_epoch = str(int(start_of_day.timestamp()))

print(start_of_day_epoch, end_of_day_epoch)


1680040801 1680127199


In [6]:
## the function result_match take a list of summoners and return a list of the results of their SOLOQ of the day 
def result_match (summoner_list):

    for summoner in summoner_list:
        w = 0
        l = 0 
        wr = 0
        winrate = []
        data_summoner = requests.get('https://euw1.api.riotgames.com/lol/summoner/v4/summoners/by-name/'+ summoner +'?api_key='+api_key).json()
        if "id" in data_summoner:
            puuid_summoner = data_summoner.get("puuid")
            
            ## with the id, the end of the day, and the start of the day we can check if the account has done any SOLOQ for the day
            
            matches_id = requests.get('https://europe.api.riotgames.com/lol/match/v5/matches/by-puuid/'+ puuid_summoner +'/ids?queue='+'420'+'&startTime='+ start_of_day_epoch +'&endTime='+ end_of_day_epoch +'&count=20&api_key='+api_key).json()

            ## check if there is any match
            
            if len(matches_id) != 0:  
                for match_id in matches_id:
                    
                    ## check the result of the game for the summoner
                    
                    match_info = requests.get("https://europe.api.riotgames.com/lol/match/v5/matches/"+ match_id +"?api_key=" + api_key).json()
                    for player in (match_info['info']['participants']):
                        if player['puuid'] == puuid_summoner:
                            if player['win'] == True:
                                w += 1
                            else:
                                l += 1
                wr = (w / (w + l)) * 100
                winrate.append( "{:.1f}%".format(wr))
                winrate.append( w + l)
        else:
            winrate.append(None) 
            winrate.append(summoner)           
        return(winrate)

result_match(summoner_list)

[]

In [7]:
#ajout des données de rank des compte dans une base de données

import sqlite3

def insert_into_leaderboard(SummonerName, Tier, Rank, LP):
  try:
    conn = sqlite3.connect('leaderboard.db')
    cur = conn.cursor()
    print("Connexion réussie à SQLite")
    sql = "INSERT INTO leaderboard (SummonerName, Tier, Rank, LP) VALUES (?, ?, ?, ?)"
    value = (SummonerName, Tier, Rank, LP)
    cur.execute(sql, value)
    conn.commit()
    print(f"Enregistrement inséré avec succès dans la table leaderboard pour le compte {SummonerName}")
    cur.close()
    conn.close()
    print("Connexion SQLite est fermée")
  except sqlite3.Error as error:
    print("Erreur lors de l'insertion dans la table leaderboard", error)

for summoner in list_summoner_rank:
    SummonerName, Tier, Rank, LP = summoner
    insert_into_leaderboard(SummonerName, Tier, Rank, LP)

Connexion réussie à SQLite
Erreur lors de l'insertion dans la table leaderboard no such table: leaderboard
Connexion réussie à SQLite
Erreur lors de l'insertion dans la table leaderboard no such table: leaderboard
Connexion réussie à SQLite
Erreur lors de l'insertion dans la table leaderboard no such table: leaderboard
Connexion réussie à SQLite
Erreur lors de l'insertion dans la table leaderboard no such table: leaderboard
Connexion réussie à SQLite
Erreur lors de l'insertion dans la table leaderboard no such table: leaderboard
Connexion réussie à SQLite
Erreur lors de l'insertion dans la table leaderboard no such table: leaderboard
Connexion réussie à SQLite
Erreur lors de l'insertion dans la table leaderboard no such table: leaderboard
Connexion réussie à SQLite
Erreur lors de l'insertion dans la table leaderboard no such table: leaderboard


In [8]:
#suppresion de la table

try:
   conn = sqlite3.connect('leaderboard.db')
   cur = conn.cursor()
   print("Connexion réussie à SQLite")
   cur.execute("delete from leaderboard")
   conn.commit()
   print("leaderboard supprimé")
   cur.close()
   conn.close()
   print("Connexion SQLite est fermée")
except sqlite3.Error as error:
   print("Erreur ", error)

Connexion réussie à SQLite
Erreur  no such table: leaderboard


In [9]:
#sorting de la liste

try:
   conn = sqlite3.connect('leaderboard.db')
   cur = conn.cursor()
   print("Connexion réussie à SQLite")
   cur.execute("
SELECT *FROM data_ranking ORDER BY CASE 
WHEN Rank = 'CHALLENGER' THEN 1 
WHEN Rank = 'GRANDMASTER' THEN 2 
WHEN Rank = 'MASTER' THEN 3 
WHEN Rank = 'DIAMOND' THEN 4 
WHEN Rank = 'PLATINIUM' THEN 5 
WHEN Rank = 'GOLD' THEN 6 
WHEN Rank = 'SILVER' THEN 7 
WHEN Rank = 'BRONZE' THEN 8 
WHEN Rank = 'IRON' THEN 9 
ELSE 10 END
,Tier asc,lp desc;
")
   conn.commit()
   print("leaderboard sorted")
   cur.close()
   conn.close()
   print("Connexion SQLite est fermée")
except sqlite3.Error as error:
   print("Erreur ", error)

SyntaxError: unterminated string literal (detected at line 7) (2102474543.py, line 7)

In [ ]:

import requests


api_key = "RGAPI-6adfe7bd-a7a9-42f9-b5ed-87ea29024f5f"


# RECUPERATION DE LA BASE DE DONNEES OFFICIELLE DES CHAMPIONS
# On le fait ici, parce que ca ne dépend pas des informations du joueur

# Récupère les versions du jeu
version_patch = requests.get("https://ddragon.leagueoflegends.com/api/versions.json").json()
# Récupère la dernière version du jeu
version_patch = version_patch[0]
# Récupère les données des champions en francais a partir de la derniere version du jeu
# Stocke les données des champions dans la variable data_full_champion
data_full_champion = requests.get('https://ddragon.leagueoflegends.com/cdn/'+ version_patch + '/data/fr_FR/champion.json').json()
# Stocke les données des champions dans la variable data_champion_by_name
data_champion_by_name = data_full_champion["data"]



# Definition de la fonction get_champion_name_by_id qui récupère le nom d'un champion à partir de son ID
def get_champion_name_by_id(id):
    # utilisation de la variable globale data_champion_by_name
    global data_champion_by_name
    # On parcourt la liste des champions
    for name in data_champion_by_name:
        # On vérifie si l'id du champion correspond à l'id du champion recherché
       if data_champion_by_name[name]['key']== str(id):
            # Si oui, on retourne le nom du champion
            return name
    return False


# Definition de la fonction afficher_champions
def afficher_champions(summoner_name, champions_count):

    # Utilisation de la variable globale api_key
    global api_key

    data_summoner = requests.get(
        f"https://euw1.api.riotgames.com/lol/summoner/v4/summoners/by-name/{summoner_name}?api_key={api_key}"
    ).json()


    if type(data_summoner) == dict and "status" in data_summoner:
        if 403 == data_summoner["status"]["status_code"]:
            print("API key expired.")
            exit()
        elif "id" not in data_summoner:
            print("Erreur, la requête n'a pas retourné d'id")
            exit()

    summoner_id = data_summoner.get("id")

    champions = requests.get(
        f"https://euw1.api.riotgames.com/lol/champion-mastery/v4/champion-masteries/by-summoner/{summoner_id}?api_key={api_key}"
    ).json()

    if type(champions) == dict:
        if 403 == champions["status"]["status_code"]:
            print("API key expired.")
            exit()
    elif type(champions) != list:
        print("Erreur, la variable champions n'est pas une liste")
        exit()
        

    champions_data = []

    # On verifie que le nombre de champions demandé n'est pas superieur au nombre de champions du joueur
    # Si tel est le cas, on ne demande que le nombre de champions du joueur

    # len = length
    # len(champions) -> récupération de la longueur de la liste champions
    if champions_count > len(champions):
        champions_count = len(champions)


    for champion in champions[:int(champions_count)]:

        # On crée un dictionnaire pour stocker les données du champion
        champion_formatted_data = {}

        # On récupère le nom du champion à partir de son ID
        champion_name = get_champion_name_by_id(champion["championId"])

        # On vérifie si le nom du champion a été trouvé
        if champion_name == False:
            print("Champion's name not found")
            champion_formatted_data["name"] = champion["championId"]
        else:
            champion_formatted_data["name"] = champion_name

        # On ajoute les autres données du champion
        champion_formatted_data["points"] = champion["championPoints"]
        champion_formatted_data["tokens"] = champion["tokensEarned"]

        # On ajoute le dictionnaire du champion à la liste des champions
        champions_data.append(champion_formatted_data)


    champions_sorted_by_points = champions_data.copy()
    champions_sorted_by_points.sort(key=lambda champion : champion["points"], reverse=True)
    print(champions_sorted_by_points)
    
    
afficher_champions("SaintLoutr",5)



[{'name': 'Velkoz', 'points': 62379, 'tokens': 0}, {'name': 'Ezreal', 'points': 54455, 'tokens': 3}, {'name': 'Kaisa', 'points': 51950, 'tokens': 2}, {'name': 'Vayne', 'points': 42503, 'tokens': 3}, {'name': 'MissFortune', 'points': 33313, 'tokens': 1}]



















[{'name': 'Velkoz', 'points': 62379, 'tokens': 0}, {'name': 'Ezreal', 'points': 54455, 'tokens': 3}, {'name': 'Kaisa', 'points': 51950, 'tokens': 2}, {'name': 'Vayne', 'points': 42503, 'tokens': 3}, {'name': 'MissFortune', 'points': 33313, 'tokens': 1}]


In [26]:
['Haukeas Grémory', 'Silver', 3, 10],['Haukeas Grémory', 'SILVER', 4, 90]
['Sam', 'GOLD', 4, 23],['Sam', 'GOLD', 4, 0]
['ThePr1ncePotato', 'BRONZE', 3, 98],['ThePr1ncePotato', 'BRONZE', 2, 22]

def datas(old_data, new_data):
    print(old,new)
    if old[2] > new[2]:
        dif = old[2]-new[2]
        if dif > 1:
            diff = (new[3]+100)-old[3]
        else:
            diff = (new[3]+100)-old[3]
        print(dif,diff)
        
    if old[2] < new[2]:
        dif = new[2]-old[2]
        diff = new[3]-(old[3]+100)
        print(dif,diff)
    
    if old[2] == new[2]:
        dif = old[2]-new[2]
        diff = new[3]-old[3]
        print(dif, diff)
        
        new_data[3] - old_data[3] + (new_data[2] - old_data[2]) * 100
        

        


datas(['Haukeas Grémory', 'Silver', 3, 10],['Haukeas Grémory', 'SILVER', 4, 90])
datas(['Sam', 'GOLD', 4, 23],['Sam', 'GOLD', 4, 20])
datas(['ThePr1ncePotato', 'BRONZE', 3, 98],['ThePr1ncePotato', 'BRONZE', 2, 20])
datas(['ThePr1ncePotato', 'BRONZE', 3, 98],['ThePr1ncePotato', 'BRONZE', 1, 20])
    

['Haukeas Grémory', 'Silver', 3, 10] ['Haukeas Grémory', 'SILVER', 4, 90]
1 -20
['Sam', 'GOLD', 4, 23] ['Sam', 'GOLD', 4, 20]
0 -3
['ThePr1ncePotato', 'BRONZE', 3, 98] ['ThePr1ncePotato', 'BRONZE', 2, 20]
1 22
['ThePr1ncePotato', 'BRONZE', 3, 98] ['ThePr1ncePotato', 'BRONZE', 1, 20]
2 22


In [85]:
def datas(old_data, new_data):
    old_tier, old_rank, old_lp = old_data[1:]
    new_tier, new_rank, new_lp = new_data[1:]

    lp_change = new_lp - old_lp

    if (old_rank > new_rank) and (old_tier == new_tier) :
        rank_diff = old_rank - new_rank
        lp_change += 100*rank_diff
        
    if (old_rank < new_rank) and (old_tier == new_tier) :
        rank_diff = new_rank - old_rank 
        lp_change -= 100*rank_diff
        
    if old_tier != new_tier:
        lp_change = 0
        return lp_change

    return lp_change

print(datas(['Sam', 'GOLD', 4, 23],['Sam', 'GOLD', 3, 20])) # Output: -3
print(datas(['ThePr1ncePotato', 'BRONZE', 3, 98],['ThePr1ncePotato','BRONZE', 2 ,20])) # Output:22
print(datas(['Haukeas Grémory','SILVER' ,3 ,10] , ['Haukeas Grémory' ,'SILVER' ,4 ,90])) # Output:20
print(datas(['Haukeas Grémory','SILVER' ,1 ,90] , ['Haukeas Grémory' ,'GOLD' ,4 ,10])) # Output:120

97
22
-20
0
